In [117]:
import scipy.io
import numpy as np
from random import shuffle
import random
import spectral
import scipy.ndimage
from skimage.transform import rotate
import os
import patch_size
%matplotlib inline

In [109]:
DATA_PATH = os.path.join(os.getcwd(),"Data")

In [81]:
input_m = np.load('input_m.npy')
target_m = np.load('target_m.npy')
input_mat = np.load('Input.npy')
target_mat = np.load('GroundTruth_CMF2.npy')

for i in range(0,target_mat.shape[0]):
    for j in range(0,target_mat.shape[1]):
        if(target_mat[i][j]==0):
            target_mat[i][j]=2

In [82]:
HEIGHT = input_mat.shape[0]
WIDTH = input_mat.shape[1]
BAND = input_mat.shape[2]
PATCH_SIZE = patch_size.patch_size
TRAIN_PATCH,TRAIN_LABELS,TEST_PATCH,TEST_LABELS = [],[],[],[]
CLASSES = [] 
COUNT = 200 #Number of patches of each class
OUTPUT_CLASSES = 2
TEST_FRAC = 0.25 #Fraction of data to be used for testing

In [83]:
PATCH_SIZE

1

In [84]:
input_mat = input_mat.astype(float)
input_mat -= np.min(input_mat)
input_mat /= np.max(input_mat)

In [85]:
MEAN_ARRAY = np.ndarray(shape=(BAND,),dtype=float)
for i in range(BAND):
    MEAN_ARRAY[i] = np.mean(input_mat[:,:,i])

In [86]:
def Patch(height_index,width_index):
    transpose_array = np.transpose(input_mat,(2,0,1))
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = transpose_array[:, height_slice, width_slice]
    mean_normalized_patch = []
    for i in range(patch.shape[0]):
        mean_normalized_patch.append(patch[i] - MEAN_ARRAY[i]) 
    
    return np.array(mean_normalized_patch)

In [88]:
for i in range(OUTPUT_CLASSES):
    CLASSES.append([])
for i in range(HEIGHT - PATCH_SIZE + 1):
    for j in range(WIDTH - PATCH_SIZE + 1):
        curr_inp = Patch(i,j)
        curr_tar = target_mat[i + int((PATCH_SIZE - 1)/2), j + int((PATCH_SIZE - 1)/2)]
        if(curr_tar!=0):
            CLASSES[int(curr_tar-1)].append(curr_inp)

In [95]:
print(len(CLASSES[1]))

81088


In [90]:
for c in range(OUTPUT_CLASSES): #for each class
    class_population = len(CLASSES[c])
    test_split_size = int(class_population*TEST_FRAC)
        
    patches_of_current_class = CLASSES[c]
    shuffle(patches_of_current_class)
    
    #Make training and test splits
    TRAIN_PATCH.append(patches_of_current_class[:-test_split_size])
        
    TEST_PATCH.extend(patches_of_current_class[-test_split_size:])
    TEST_LABELS.extend(np.full(test_split_size, c, dtype=int))

In [118]:
#for c in TRAIN_PATCH:
    #print(len(c))

In [97]:
for k in range(OUTPUT_CLASSES):
    if(len(TRAIN_PATCH[k])<COUNT):
        tmp = TRAIN_PATCH[k]
        for j in range(int(COUNT/len(TRAIN_PATCH[k]))):
            shuffle(TRAIN_PATCH[k])
            TRAIN_PATCH[k] = TRAIN_PATCH[k] + tmp
    shuffle(TRAIN_PATCH[k])
    TRAIN_PATCH[k] = TRAIN_PATCH[k][:COUNT]

In [115]:
for c in TRAIN_PATCH:
    print(len(c))

192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192
192


In [99]:
TRAIN_PATCH = np.asarray(TRAIN_PATCH)

print(TRAIN_PATCH)

[[[[[ 0.01550085]]

   [[ 0.01412903]]

   [[ 0.0148088 ]]

   ...

   [[ 0.00444856]]

   [[ 0.00429497]]

   [[ 0.00464316]]]


  [[[ 0.0128343 ]]

   [[ 0.01172014]]

   [[ 0.01196891]]

   ...

   [[ 0.0048526 ]]

   [[ 0.00563197]]

   [[ 0.00553453]]]


  [[[ 0.0060142 ]]

   [[ 0.0055355 ]]

   [[ 0.00595956]]

   ...

   [[ 0.004705  ]]

   [[ 0.00477414]]

   [[ 0.00456752]]]


  ...


  [[[ 0.00405876]]

   [[ 0.00373463]]

   [[ 0.00378164]]

   ...

   [[ 0.0036684 ]]

   [[ 0.0029782 ]]

   [[ 0.00409937]]]


  [[[ 0.01820933]]

   [[ 0.01675482]]

   [[ 0.01668349]]

   ...

   [[ 0.00748759]]

   [[ 0.00625092]]

   [[ 0.00583636]]]


  [[[ 0.00487714]]

   [[ 0.00487198]]

   [[ 0.00485003]]

   ...

   [[ 0.00200159]]

   [[ 0.00147819]]

   [[ 0.002087  ]]]]



 [[[[-0.00028269]]

   [[-0.00030771]]

   [[-0.00031103]]

   ...

   [[ 0.00141622]]

   [[ 0.0010801 ]]

   [[ 0.0001003 ]]]


  [[[ 0.00084556]]

   [[ 0.00080415]]

   [[ 0.00074447]]

   ...

   [[ 0.0015

In [101]:
TRAIN_PATCH = TRAIN_PATCH.reshape((-1,192,PATCH_SIZE,PATCH_SIZE))

In [102]:
TRAIN_LABELS = np.array([])
for l in range(OUTPUT_CLASSES):
    TRAIN_LABELS = np.append(TRAIN_LABELS, np.full(COUNT, l, dtype=int))

In [103]:
print(len(TEST_PATCH))
print(len(TRAIN_PATCH))

20988
400


Save the patches in segments
=================================

1. Training data
----------------

In [110]:
for i in range(int(len(TRAIN_PATCH)/(COUNT*2))):
    train_dict = {}
    start = i * (COUNT*2)
    end = (i+1) * (COUNT*2)
    file_name = 'Train_'+str(PATCH_SIZE)+'_'+str(i+1)+'.mat'
    train_dict["train_patch"] = TRAIN_PATCH[start:end]
    train_dict["train_labels"] = TRAIN_LABELS[start:end]
    scipy.io.savemat(os.path.join(DATA_PATH, file_name),train_dict)

2. Test data
-------------

In [111]:
for i in range(int(len(TEST_PATCH)/(COUNT*2))):
    test_dict = {}
    start = i * (COUNT*2)
    end = (i+1) * (COUNT*2)
    file_name = 'Test_'+str(PATCH_SIZE)+'_'+str(i+1)+'.mat'
    test_dict["test_patch"] = TEST_PATCH[start:end]
    test_dict["test_labels"] = TEST_LABELS[start:end]
    scipy.io.savemat(os.path.join(DATA_PATH, file_name),test_dict)

In [112]:
len(TRAIN_PATCH)/(COUNT*2)


1.0

In [113]:
np.save('LATEST_CMF.npy',target_mat)